In [1]:
from array import array
from skmultiflow.data import RegressionGenerator
from sail.models.torch.tcn import TCNRegressor
#from sail.models.torch.tcn import TCN
import numpy as np
import torch

In [2]:
n_samples = 6000
n_features = 12
stream = RegressionGenerator(random_state=1,
                             n_samples=n_samples,
                             n_features=n_features)
learner_tcn = TCNRegressor(n_features, 1)


In [3]:
cnt = 0
y_true = array('d')
y_pred = array('d') 
index = []
i = 0
wait_samples = 30
while cnt < n_samples and stream.has_more_samples():
    X, y = stream.next_sample(batch_size=wait_samples)
    y = np.array(y)
    X = np.array(X)
    X = X.reshape(X.shape[0], X.shape[1], -1)
    y = y.reshape(y.shape[0], -1)
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    # Test every n samples
    if (cnt % wait_samples == 0) & (cnt != 0):
        y_true.append(y[0])
        y_pred1 = learner_tcn.predict(X)[0][0]
        y_pred.append(y_pred1)
        index.append(i)
        i = i + 1
    learner_tcn.partial_fit(X, y)
    #learner_tcn.predict(X)
    cnt += 1

  epoch    train_loss     dur
-------  ------------  ------
      1    36861.3730  0.0640
      2    30080.5003  0.0400
      3    22639.8799  0.0440
      4    13900.8678  0.0380
      5    23738.1152  0.0440
      6    28056.0521  0.0410
      7    25443.7734  0.0420
      8    18505.4639  0.0430
      9    21298.7930  0.0400
     10    38952.5944  0.0370
     11    33982.8021  0.0420
     12    26913.8659  0.0420
     13    33352.4271  0.0420
     14    20728.8174  0.0410
     15    35340.3346  0.0440
     16    24270.5872  0.0410
     17    18023.1328  0.0430
     18    25873.4258  0.0410
     19    16617.1751  0.0450
     20    33355.7038  0.0410
     21    32119.4544  0.0400
     22    38390.3568  0.0420
     23    28266.1797  0.0400
     24    37191.2135  0.0420
     25    24587.6357  0.0410
     26    36838.6237  0.0390
     27    15796.1816  0.0450
     28    24403.3906  0.0390
     29    18654.2533  0.0420
     30    34941.4531  0.0390
     31    27040.9557  0.0400
     32   

In [4]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'vscode'

y_pred = y_pred.tolist()
y_true = y_true.tolist()
fig = go.Figure()

fig.add_trace(go.Scatter(x=index, y=y_pred, name='predicted',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=index, y=y_true, name='actual',
                         line=dict(color='blue', width=2)))
fig.show()